# Claude Browser Integration Demo

This notebook demonstrates how to use the `claude-browser` package to interact with Claude.ai through browser automation.

## Why Browser Automation?

- ✅ **No API key required** - Uses your existing Claude session
- ✅ **Bypasses Cloudflare** - Works where APIs fail
- ✅ **Full Claude capabilities** - Access to all features
- ✅ **Works on Raspberry Pi** - Optimized for ARM devices

## Installation

First, install the package and its dependencies:

In [ ]:
# Install the package (uncomment if needed)
# !pip install -e .

# Install Playwright browsers (run once)
# !playwright install chromium
# !playwright install-deps

## Authentication Setup

You need to get your session cookies from Claude.ai:

1. Log into [Claude.ai](https://claude.ai) in your browser
2. Open Developer Tools (F12)
3. Go to Application → Cookies → claude.ai
4. Copy the `sessionKey` value
5. Set it below:

In [ ]:
import os

# Set your session key here (or use environment variable)
os.environ['CLAUDE_SESSION_KEY'] = 'sk-ant-sid01-...'  # Replace with your session key

# Optional: Set organization and user IDs if you have them
# os.environ['CLAUDE_ORG_ID'] = 'your-org-id'
# os.environ['CLAUDE_USER_ID'] = 'your-user-id'

## Basic Usage

Let's start with a simple example:

In [ ]:
import asyncio
from claude_browser import ClaudeBrowserClient

async def hello_claude():
    """Simple example to test the connection."""
    
    # Create client (headless=False to see the browser)
    client = ClaudeBrowserClient(headless=True)
    
    try:
        # Connect to Claude
        print("🚀 Connecting to Claude...")
        if await client.initialize():
            print("✅ Connected!")
            
            # Send a message
            response = await client.send_message("Hello! Please respond with exactly 5 words.")
            print(f"\n🤖 Claude says: {response}")
        else:
            print("❌ Failed to connect")
    finally:
        await client.close()

# Run the async function
await hello_claude()

## Interactive Conversation

Now let's have a more interactive conversation:

In [ ]:
class ClaudeChat:
    """Interactive Claude chat wrapper."""
    
    def __init__(self):
        self.client = None
    
    async def start(self):
        """Start the chat session."""
        self.client = ClaudeBrowserClient(headless=True)
        success = await self.client.initialize()
        if success:
            print("✅ Claude is ready to chat!")
        else:
            print("❌ Failed to connect to Claude")
        return success
    
    async def ask(self, question):
        """Ask Claude a question."""
        if not self.client:
            print("❌ Not connected. Run start() first.")
            return
        
        print(f"\n💬 You: {question}")
        response = await self.client.send_message(question)
        print(f"\n🤖 Claude: {response}")
        return response
    
    async def stop(self):
        """End the chat session."""
        if self.client:
            await self.client.close()
            print("\n👋 Chat ended")

# Create chat instance
chat = ClaudeChat()

In [ ]:
# Start the chat
await chat.start()

In [ ]:
# Ask a question
await chat.ask("What's the capital of France?")

In [ ]:
# Ask a follow-up question
await chat.ask("Tell me one interesting fact about it.")

In [ ]:
# More complex question
await chat.ask("Write a haiku about programming.")

In [ ]:
# End the chat
await chat.stop()

## Context Manager Example

Using a context manager for automatic cleanup:

In [ ]:
async def context_manager_example():
    """Example using context manager."""
    
    # Context manager handles initialization and cleanup
    async with ClaudeBrowserClient(headless=True) as client:
        print("✅ Connected to Claude!")
        
        # Send a message
        response = await client.send_message(
            "Explain quantum computing in one sentence."
        )
        print(f"\nClaude: {response}")
    
    print("\n✅ Automatically cleaned up!")

await context_manager_example()

## Error Handling

Demonstrating proper error handling:

In [ ]:
from claude_browser import (
    ClaudeAuthenticationError,
    ClaudeTimeoutError,
    ClaudeResponseError
)

async def safe_chat():
    """Example with error handling."""
    
    client = ClaudeBrowserClient(headless=True)
    
    try:
        await client.initialize()
        
        # Try to send a message with a short timeout
        response = await client.send_message(
            "What's 2+2?",
            timeout=5  # 5 second timeout
        )
        print(f"Claude: {response}")
        
    except ClaudeAuthenticationError:
        print("❌ Authentication failed - check your session key")
    except ClaudeTimeoutError:
        print("⏱️ Request timed out")
    except ClaudeResponseError:
        print("📝 Failed to extract response")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    finally:
        await client.close()
        print("Cleaned up")

await safe_chat()

## Performance Test

Let's measure response times:

In [ ]:
import time

async def performance_test():
    """Measure response times."""
    
    async with ClaudeBrowserClient(headless=True) as client:
        questions = [
            "What's 2+2?",
            "Name a color.",
            "Say 'hello'."
        ]
        
        print("⏱️ Performance Test\n")
        
        for question in questions:
            start = time.time()
            response = await client.send_message(question)
            elapsed = time.time() - start
            
            print(f"Q: {question}")
            print(f"A: {response}")
            print(f"Time: {elapsed:.2f} seconds\n")

await performance_test()

## Tips and Tricks

### 1. Debugging
Set `headless=False` to see the browser:
```python
client = ClaudeBrowserClient(headless=False)
```

### 2. Custom Chromium Path
For Raspberry Pi:
```python
client = ClaudeBrowserClient(
    chromium_path="/usr/bin/chromium-browser"
)
```

### 3. Longer Timeouts
For slow connections:
```python
client = ClaudeBrowserClient(timeout=60000)  # 60 seconds
```

### 4. Session Management
Keep the client alive for multiple messages:
```python
client = ClaudeBrowserClient()
await client.initialize()
# Send many messages...
await client.close()  # Only close when done
```

## Conclusion

You now have a working Claude browser integration that:
- ✅ Bypasses API limitations
- ✅ Works without API keys
- ✅ Handles ProseMirror editor correctly
- ✅ Runs on Raspberry Pi
- ✅ Provides full Claude capabilities

Remember: The key breakthrough was handling ProseMirror with:
```python
await page.keyboard.press('Control+A')
await page.keyboard.press('Backspace')
```

Happy chatting with Claude! 🤖